## Importação dos arquivos e geração dos segmentos

In [1]:
import os
import pandas as pd
import numpy as np
import pprint

# =============================================================================
# BLOCO 1: CONFIGURAÇÃO, CARREGAMENTO, DIVISÃO (80/20) E SEGMENTAÇÃO (CORRIGIDO)
# =============================================================================

# --- 1. CONFIGURAÇÕES GERAIS ---
caminho_raiz = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset' # IMPORTANTE: Verifique se este caminho está correto
params_drive_end = {'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0}
TAXA_AMOSTRAL = 12000

# Dicionários de mapeamento
mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}

# --- PARÂMETROS DE SEGMENTAÇÃO ---
tamanho_segmento = 4096
sobreposicao_percentual = 0.3
passo = int(tamanho_segmento * (1 - sobreposicao_percentual))

# --- 2. CARREGAMENTO, DIVISÃO E PROCESSAMENTO ---
dicionario_treino = {} # Dicionário para 80% dos dados normais
dicionario_teste = {} # Dicionário para 20% normais + 100% falhas reais

print(f"Iniciando a leitura e segmentação dos arquivos em '{caminho_raiz}'...")
print("Dados normais serão divididos (80% treino / 20% teste).")
print("Dados de falha real irão 100% para o teste.")

# Função auxiliar para segmentar um sinal e adicionar ao dicionário
def segmentar_e_adicionar(sinal, metadados, dicionario_alvo, chave_base):
    # Verifica se o sinal é longo o suficiente para pelo menos um segmento
    if len(sinal) < tamanho_segmento:
        # print(f"Aviso: Sinal da base '{chave_base}' muito curto ({len(sinal)} amostras) para gerar segmentos. Ignorando.")
        return 0

    num_segmentos_criados = 0
    for i, inicio in enumerate(range(0, len(sinal) - tamanho_segmento + 1, passo)):
        segmento = sinal[inicio : inicio + tamanho_segmento]
        df_segmento = pd.DataFrame({'amplitude': segmento})

        # Adiciona metadados
        df_segmento['arquivo_origem'] = metadados['nome_arquivo']
        df_segmento['rotacao_rpm'] = metadados['rpm']
        df_segmento['tipo_falha'] = metadados['tipo_falha']
        df_segmento['diametro_falha'] = metadados['diametro_falha']
        df_segmento['local_sensor'] = 'Drive End'

        chave_segmento = f"{chave_base}_seg_{i}"
        dicionario_alvo[chave_segmento] = df_segmento
        num_segmentos_criados += 1
    return num_segmentos_criados

# Loop principal pelos arquivos
for pasta_atual, _, arquivos in os.walk(caminho_raiz):
    for nome_arquivo in arquivos:
        # Processar apenas arquivos .npz
        if nome_arquivo.endswith('.npz'):
            caminho_completo = os.path.join(pasta_atual, nome_arquivo)

            # Decodificação de metadados
            nome_sem_ext = nome_arquivo.replace('.npz', '')
            partes = nome_sem_ext.split('_')
            rpm_str = partes[0]
            is_normal = 'Normal' in nome_arquivo

            metadados = {
                'nome_arquivo': nome_arquivo,
                'rpm': int(rpm_str) if rpm_str.isdigit() else 0,
                'tipo_falha': 'Normal' if is_normal else mapa_tipo_falha.get(partes[1].split('@')[0], 'Desconhecido'),
                'diametro_falha': 'N/A' if is_normal else mapa_diametro_falha.get(partes[2], 'Desconhecido')
            }

            try:
                dados_npz = np.load(caminho_completo)
                sensor_cod = 'DE' # Foco apenas no Drive End, como no seu código original

                if sensor_cod in dados_npz.files:
                    sinal_completo = dados_npz[sensor_cod].ravel()

                    if is_normal:
                        # DIVIDE O SINAL NORMAL EM 80/20
                        ponto_corte = int(len(sinal_completo) * 0.8)
                        sinal_treino = sinal_completo[:ponto_corte]
                        sinal_teste = sinal_completo[ponto_corte:]

                        chave_base_normal = f"{nome_sem_ext}_{sensor_cod}"
                        segmentar_e_adicionar(sinal_treino, metadados, dicionario_treino, f"{chave_base_normal}_treino")
                        segmentar_e_adicionar(sinal_teste, metadados, dicionario_teste, f"{chave_base_normal}_teste")

                    else:
                        # Sinais com falha (REAIS) vão inteiramente para o TESTE
                        # Lógica de chave para arquivos de falha (igual ao seu original)
                        partes_chave = nome_sem_ext.split('_')
                        partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                        chave_base_falha = "_".join(partes_chave)
                        
                        # =================================================================
                        # MUDANÇA PRINCIPAL AQUI: Envia falhas reais para o dicionario_teste
                        # =================================================================
                        segmentar_e_adicionar(sinal_completo, metadados, dicionario_teste, chave_base_falha)

            except Exception as e:
                print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

# --- Relatório Final (Atualizado para refletir a nova lógica) ---
print("\n--- Processo Concluído! ---")
print(f"Total de segmentos de TREINO (APENAS 80% normais): {len(dicionario_treino)}")
print(f"Total de segmentos de TESTE (falhas reais + 20% normais): {len(dicionario_teste)}")

if not dicionario_teste:
    print("\nAVISO: O dicionário de teste está vazio. Verifique se os arquivos 'Normal' existem e se os sinais são longos o suficiente.")

if dicionario_treino:
    # Garante que dicionário não está vazio antes de tentar acessar
    if len(dicionario_treino) > 0:
        chave_exemplo_treino = list(dicionario_treino.keys())[0]
        print(f"\nExemplo de um segmento de TREINO (chave: '{chave_exemplo_treino}'):")
        print(dicionario_treino[chave_exemplo_treino].head())
    else:
        print("\nO dicionário de TREINO está vazio.")

if dicionario_teste:
     # Garante que dicionário não está vazio antes de tentar acessar
    if len(dicionario_teste) > 0:
        chave_exemplo_teste = list(dicionario_teste.keys())[0]
        print(f"\nExemplo de um segmento de TESTE (chave: '{chave_exemplo_teste}'):")
        print(dicionario_teste[chave_exemplo_teste].head())
    else:
        print("\nO dicionário de TESTE está vazio.")

Iniciando a leitura e segmentação dos arquivos em 'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'...
Dados normais serão divididos (80% treino / 20% teste).
Dados de falha real irão 100% para o teste.

--- Processo Concluído! ---
Total de segmentos de TREINO (APENAS 80% normais): 470
Total de segmentos de TESTE (falhas reais + 20% normais): 2897

Exemplo de um segmento de TREINO (chave: '1730_Normal_DE_treino_seg_0'):
   amplitude   arquivo_origem  rotacao_rpm tipo_falha diametro_falha  \
0   0.014603  1730_Normal.npz         1730     Normal            N/A   
1   0.054449  1730_Normal.npz         1730     Normal            N/A   
2   0.107646  1730_Normal.npz         1730     Normal            N/A   
3   0.133722  1730_Normal.npz         1730     Normal            N/A   
4   0.112652  1730_Normal.npz         1730     Normal            N/A   

  local_sensor  
0    Drive End  
1    Drive End  
2    Drive End  
3    Drive End  
4    Drive End  

Exemplo de um segmento de TESTE 

## Criação do sinal sintético de falha 

### Características do sinal de falha Po, Pr, Fs, Fo

In [48]:
import numpy as np

def calculate_tandon_coefficients(fault_diameter_mm, rpm, fault_type):
    """
    Calculates Fourier coefficients (Po, Pr, Fo, Fs) for bearing analysis 
    based on the Tandon and Choudhury (1997) model.

    Parameters:
    -----------
    fault_diameter_mm : float
        Width of the defect in millimeters (b).
    rpm : float
        Shaft rotational speed in Revolutions Per Minute.
    fault_type : str
        Type of fault: 'inner', 'outer', or 'ball'.

    Returns:
    --------
    dict
        A dictionary containing:
        - 'Po': Load distribution mean coefficient (float)
        - 'Pr': Function to calculate r-th harmonic of load (callable)
        - 'Fo': Pulse mean coefficient (float)
        - 'Fs': Function to calculate s-th harmonic of pulse (callable)
        - 'frequencies': Dictionary of characteristic frequencies (Hz)
        - 'debug_info': Intermediate values (pulse width, relative velocity, etc.)
    """

    # ==========================================
    # 1. CONSTANTS & BEARING GEOMETRY (6205-2 RS)
    # ==========================================
    # Dimensions converted to meters for calculation consistency
    D_ext = 52.0 * 1e-3     # Outer Diameter
    D_int = 25.0 * 1e-3     # Inner Diameter
    W = 15.0 * 1e-3         # Width
    d = 7.94 * 1e-3         # Ball diameter (d) [cite: 1054]
    D = 39.05 * 1e-3        # Pitch diameter (D) [cite: 1054]
    Z = 9                   # Number of balls [cite: 1054]
    alpha_deg = 0.0         # Contact angle (assumed 0 for zero clearance)
    alpha = np.radians(alpha_deg)
    
    # Load parameters
    radial_load_kg = 100
    radial_load_N = radial_load_kg * 9.80665 # Convert kg to Newtons
    
    # Pulse Height (K) - Assumed 1 for normalized analysis as is standard in Tandon's results
    K = 1.0 

    # ==========================================
    # 2. KINEMATIC CALCULATIONS
    # ==========================================
    # Shaft angular velocity (rad/s)
    w_s = (2 * np.pi * rpm) / 60.0  
    f_s = rpm / 60.0 # Shaft frequency Hz

    # Cage angular velocity (rad/s) - Eq in Appendix I [cite: 1859]
    # w_c = (w_s / 2) * (1 - (d/D) * cos(alpha))
    w_c = (w_s / 2.0) * (1.0 - (d / D) * np.cos(alpha))

    # Ball Spin angular velocity (rad/s) - Eq in Appendix I [cite: 1860]
    w_b = (D * w_s / (2.0 * d)) * (1.0 - (d**2 / D**2) * (np.cos(alpha)**2))

    # Determine Characteristic Defect Frequency (rad/s) based on fault type [cite: 1861, 1863, 1864]
    if fault_type.lower() == 'inner':
        # Inner race defect frequency: Z(w_s - w_c)
        w_defect = Z * (w_s - w_c) 
    elif fault_type.lower() == 'outer':
        # Outer race defect frequency: Z * w_c
        w_defect = Z * w_c
    elif fault_type.lower() == 'ball':
        # Ball defect frequency: 2 * w_b
        w_defect = 2.0 * w_b
    else:
        raise ValueError("fault_type must be 'inner', 'outer', or 'ball'")

    # Period of the pulse (T)
    # Note: For inner/outer race, T is time between ball passes. 
    # For ball fault, T is time between impacts on races.
    T = 2 * np.pi / w_defect

    # ==========================================
    # 3. PULSE WIDTH & DUTY CYCLE
    # ==========================================
    # Relative Velocity (v_r) - Tandon Eq. 11 [cite: 1462]
    # v_r = (D * w_s / 4) * (1 - (d^2 / D^2) * cos^2(alpha))
    # Note: This specific formula is unique to Tandon's relative velocity calculation.
    v_r = (D * w_s / 4.0) * (1.0 - (d**2 / D**2) * (np.cos(alpha)**2))
    
    # Defect width (b) in meters
    b = fault_diameter_mm * 1e-3

    # Pulse Width (Delta T) - Tandon Eq. 12 [cite: 1463]
    delta_T = b / v_r

    # Duty Cycle (m) - Appendix II [cite: 1874]
    m = delta_T / T

    # Safety check: duty cycle cannot exceed 1
    if m > 1.0:
        print(f"Warning: Defect size is too large for the RPM. Duty cycle {m:.2f} clipped to 1.0")
        m = 1.0

    # ==========================================
    # 4. LOAD DISTRIBUTION COEFFICIENTS (Po, Pr)
    # ==========================================
    # Max Load (P_max) - Stribeck Equation (Harris 1966 via Tandon Eq 3.1) [cite: 1208]
    # P_max = (4.37 * F_r) / (Z * cos(alpha))
    P_max = (4.37 * radial_load_N) / (Z * np.cos(alpha))

    # Load Zone Parameters for Zero Clearance
    # epsilon = 0.5 
    # phi_1 = pi/2 (Load zone extent)
    epsilon = 0.5
    phi_1 = np.pi / 2.0
    
    # Ball bearing load-deflection exponent (n)
    n_exp = 1.5 

    # Coefficients A0, A1, A2, A3 - Tandon Eq. 8b [cite: 1443]
    # Note: These are derived from binomial expansion of load distribution
    A0 = 1.0 - (n_exp / (2*epsilon)) + (n_exp*(n_exp-1) / (8*epsilon**2)) * 1.5 - (n_exp*(n_exp-1)*(n_exp-2) / (48*epsilon**3)) * 2.5
    A1 = (n_exp / (2*epsilon)) - (n_exp*(n_exp-1) / (8*epsilon**2)) * 2.0 + (n_exp*(n_exp-1)*(n_exp-2) / (48*epsilon**3)) * 3.75
    A2 = (n_exp*(n_exp-1) / (8*epsilon**2)) * 0.5 - (n_exp*(n_exp-1)*(n_exp-2) / (48*epsilon**3)) * 1.5
    A3 = (n_exp*(n_exp-1)*(n_exp-2) / (48*epsilon**3)) * 0.25
    
    # Store A coefficients in a list for easier summation in Pr
    A_coeffs = [A0, A1, A2, A3]

    # Calculate Po - Tandon Eq. 9 [cite: 1446]
    # Po = (P_max / pi) * [A0*phi1 + A1*sin(phi1) + (A2/2)*sin(2*phi1) + (A3/3)*sin(3*phi1)]
    term_Po = A0 * phi_1 + A1 * np.sin(phi_1) + (A2/2.0) * np.sin(2.0*phi_1) + (A3/3.0) * np.sin(3.0*phi_1)
    Po = (P_max / np.pi) * term_Po

    # Define function for Pr (Load Harmonics) - Tandon Eq. 10 [cite: 1446]
    def get_Pr(r):
        """Calculates the r-th Fourier coefficient for Load P."""
        if r == 0:
            return Po
        
        # Term 1: 2*A0/r * sin(r*phi1)
        term1 = (2.0 * A0 / r) * np.sin(r * phi_1)
        
        # Term 2: Summation for l=1 to 3
        term2 = 0.0
        for l in range(1, 4): # l = 1, 2, 3
            Al = A_coeffs[l]
            
            # Handling singularity if r == l (though r is usually integer > 0)
            if r == l:
                # Limit sin(x)/x as x->0 is 1. But here denominator is (r-l).
                # Term becomes phi1 * cos(0) = phi1
                val_minus = phi_1 
            else:
                val_minus = np.sin((r - l) * phi_1) / (r - l)
                
            val_plus = np.sin((r + l) * phi_1) / (r + l)
            
            term2 += Al * (val_plus + val_minus)
            
        return (P_max / np.pi) * (term1 + term2)

    # ==========================================
    # 5. PULSE SHAPE COEFFICIENTS (Fo, Fs)
    # ==========================================
    # Rectangular Pulse Model - Appendix II [cite: 1872, 1873]
    
    # Fo = K * m [cite: 1873]
    Fo = K * m

    # Define function for Fs (Pulse Harmonics) - Appendix II [cite: 1875]
    def get_Fs(s):
        """Calculates the s-th Fourier coefficient for Pulse F (Rectangular)."""
        if s == 0:
            return Fo
        
        # Fs = (2*K / (pi*s)) * sin(pi*s*m)
        return (2.0 * K / (np.pi * s)) * np.sin(np.pi * s * m)

    # ==========================================
    # 6. PACKAGING RESULTS
    # ==========================================
    return {
        "Po": Po,
        "Pr": get_Pr,  # Callable function: Pr(r)
        "Fo": Fo,
        "Fs": get_Fs,  # Callable function: Fs(s)
        "frequencies": {
            "shaft_freq_hz": w_s / (2*np.pi),
            "cage_freq_hz": w_c / (2*np.pi),
            "defect_freq_hz": w_defect / (2*np.pi)
        },
        "debug_info": {
            "v_r_m_s": v_r,
            "delta_T_sec": delta_T,
            "period_T_sec": T,
            "duty_cycle_m": m,
            "P_max_N": P_max,
            "load_coeffs_A": A_coeffs
        }
    }

# ==========================================
# EXAMPLE USAGE
# ==========================================

# Inputs
fault_mm = 1.5   # 1.5 mm fault diameter
rpm_val = 1500   # 1500 RPM
f_type = 'inner' # Inner race fault

# Calculate coefficients
coeffs = calculate_tandon_coefficients(fault_mm, rpm_val, f_type)

# 1. Inspect Scalar Mean Values
print(f"--- Results for {f_type.upper()} Fault ---")
print(f"Po (Static Load Component): {coeffs['Po']:.2f} N")
print(f"Fo (Pulse Mean Component):  {coeffs['Fo']:.4f}")

# 2. Calculate First 5 Harmonics for Load (Pr) and Pulse (Fs)
print("\n--- Harmonics (First 5) ---")
print(f"{'Harmonic (n)':<15} | {'Pr (Load) [N]':<15} | {'Fs (Pulse)':<15}")
print("-" * 50)
for n in range(1, 6):
    pr_val = coeffs['Pr'](n)
    fs_val = coeffs['Fs'](n)
    print(f"{n:<15} | {pr_val:<15.2f} | {fs_val:<15.4f}")

# 3. Debug Information (Verify physics)
print("\n--- Debug Info ---")
debug = coeffs['debug_info']
print(f"Max Load (Pmax): {debug['P_max_N']:.2f} N")
print(f"Pulse Width:     {debug['delta_T_sec']*1000:.4f} ms")
print(f"Duty Cycle (m):  {debug['duty_cycle_m']:.4f}")
print(f"Relative Vel (vr): {debug['v_r_m_s']:.2f} m/s")

--- Results for INNER Fault ---
Po (Static Load Component): 131.02 N
Fo (Pulse Mean Component):  0.1381

--- Harmonics (First 5) ---
Harmonic (n)    | Pr (Load) [N]   | Fs (Pulse)     
--------------------------------------------------
1               | 217.49          | 0.2677         
2               | 116.22          | 0.2429         
3               | 25.33           | 0.2046         
4               | -12.45          | 0.1570         
5               | -7.04           | 0.1052         

--- Debug Info ---
Max Load (Pmax): 476.17 N
Pulse Width:     1.0203 ms
Duty Cycle (m):  0.1381
Relative Vel (vr): 1.47 m/s


### Cálculo das frequencias naturais dos dois aneis


In [53]:
import numpy as np
import pandas as pd

def get_bearing_natural_frequencies():
    """
    Calculates natural frequencies and mass for the Outer and Inner races
    of a 6205-2 RS bearing using Tandon's Equations 1 & 15.
    
    Returns:
        df_modes (pd.DataFrame): Dataframe containing frequencies, mass, 
                                 and geometry for each mode.
    """
    # ---------------------------------------------------------
    # 1. Fixed Parameters (User Provided)
    # ---------------------------------------------------------
    D_ext = 52.0 * 1e-3     # Outer Diameter (m)
    D_int = 25.0 * 1e-3     # Inner Diameter (m)
    W = 15.0 * 1e-3         # Width (m)
    d = 7.94 * 1e-3         # Ball diameter (m)
    D = 39.05 * 1e-3        # Pitch diameter (m)
    Z = 9                   # Number of balls
    
    # Material Properties (Standard Bearing Steel)
    rho_vol = 7850          # Density (kg/m^3)
    E = 2.07e11             # Young's Modulus (Pa)

    # ---------------------------------------------------------
    # 2. Geometry Approximation (Rectangular Cross-Section)
    # ---------------------------------------------------------
    # We estimate the effective thickness (h) for the "equivalent rectangular ring"
    # assumption used by Tandon.
    
    # --- Outer Race Geometry ---
    # Max thickness (at shoulder) approx: (D_ext - Pitch) / 2
    h_out_max = (D_ext - D) / 2.0
    # Min thickness (at groove bottom) approx: (D_ext - (Pitch + Ball)) / 2
    h_out_min = (D_ext - (D + d)) / 2.0
    # Effective thickness (average)
    h_outer = (h_out_max + h_out_min) / 2.0
    # Neutral axis radius (a)
    a_outer = (D_ext / 2.0) - (h_outer / 2.0)

    # --- Inner Race Geometry ---
    # Max thickness approx: (Pitch - D_int) / 2
    h_in_max = (D - D_int) / 2.0
    # Min thickness approx: ((Pitch - Ball) - D_int) / 2
    h_in_min = ((D - d) - D_int) / 2.0
    # Effective thickness
    h_inner = (h_in_max + h_in_min) / 2.0
    # Neutral axis radius (a)
    a_inner = (D_int / 2.0) + (h_inner / 2.0)

    # ---------------------------------------------------------
    # 3. Calculation Function
    # ---------------------------------------------------------
    def calc_race_modes(race_name, h, a, n_modes=1000):
        results = []
        
        # Section Properties
        I = (W * h**3) / 12.0       # Area Moment of Inertia (m^4)
        A_cs = W * h                # Cross-sectional Area (m^2)
        rho_lin = rho_vol * A_cs    # Mass per unit length (kg/m)
        
        # Generalized Mass (M_i) - Equation 15 (Tandon)
        # "Equals the actual mass of the ring" and is constant for all modes.
        M_i = 2 * np.pi * rho_lin * a 
        
        for i in range(2, n_modes + 1): # Modes i = 2, 3, 4, ...
            
            # Natural Frequency - Equation 1 (Tandon)
            # w_i = (i(i^2-1) / sqrt(1+i^2)) * sqrt(EI / rho a^4)
            term_mode = (i * (i**2 - 1)) / np.sqrt(1 + i**2)
            term_phys = np.sqrt((E * I) / (rho_lin * a**4))
            
            w_n_rad = term_mode * term_phys
            f_n_hz = w_n_rad / (2 * np.pi)
            
            results.append({
                'Race': race_name,
                'Mode_i': i,
                'Freq_Hz': round(f_n_hz, 2),
                'Freq_rad_s': round(w_n_rad, 2),
                'Mass_kg': round(M_i, 5),
                'I_m4': I,
                'radius_a_m': a
            })
        return results

    # ---------------------------------------------------------
    # 4. Execute & Store
    # ---------------------------------------------------------
    data_outer = calc_race_modes("Outer", h_outer, a_outer)
    data_inner = calc_race_modes("Inner", h_inner, a_inner)
    
    df = pd.DataFrame(data_outer + data_inner)
    return df

# Run and Display
df_natural_frequencies = get_bearing_natural_frequencies()
print(df_natural_frequencies[['Race', 'Mode_i', 'Freq_Hz', 'Mass_kg']])

       Race  Mode_i       Freq_Hz  Mass_kg
0     Outer       2  5.037120e+03  0.07891
1     Outer       3  1.424713e+04  0.07891
2     Outer       4  2.731762e+04  0.07891
3     Outer       5  4.417847e+04  0.07891
4     Outer       6  6.480889e+04  0.07891
...     ...     ...           ...      ...
1993  Inner     996  5.228638e+09  0.05601
1994  Inner     997  5.239142e+09  0.05601
1995  Inner     998  5.249657e+09  0.05601
1996  Inner     999  5.260183e+09  0.05601
1997  Inner    1000  5.270719e+09  0.05601

[1998 rows x 4 columns]


## Amplitude de falhas modelagem de Tandon

### Outer race

In [49]:
import numpy as np
import pandas as pd

def calcular_espectro_outer_race(fault_diameter_mm, rpm, max_harmonics=10):
    """
    Calcula o espectro de aceleração (teórico) para falha na Pista Externa (Outer Race)
    localizada no centro da zona de carga, baseando-se na Eq. 20 de Tandon.
    
    Dependências:
    - calculate_tandon_coefficients (já definida no notebook)
    - get_bearing_natural_frequencies (já definida no notebook)
    """
    
    # 1. Obter Coeficientes de Tandon (Po, Fs, Pmax, etc.)
    # Chamamos sua função passando 'outer' para configurar as frequências corretamente
    tandon_data = calculate_tandon_coefficients(fault_diameter_mm, rpm, fault_type='outer')
    
    # Extração de parâmetros críticos retornados pela sua função
    P_max = tandon_data['debug_info']['P_max_N']  # Carga máxima (centro da zona)
    func_Fs = tandon_data['Fs']                    # Função callable para harmônicos do pulso
    fc_Hz = tandon_data['frequencies']['cage_freq_hz'] # Frequência da gaiola
    
    # 2. Obter Propriedades Modais (Frequências Naturais e Massas)
    df_nat_freq = get_bearing_natural_frequencies()
    # Filtramos apenas para o anel externo (Outer)
    df_outer = df_nat_freq[df_nat_freq['Race'] == 'Outer']
    
    # Constante Z (Número de esferas) - Hardcoded em 9 conforme seus outros códigos
    Z = 9 
    
    # Listas para armazenar o espectro
    freqs_hz = []
    ampls_accel = []
    
    # 3. Loop pelos Harmônicos da BPFO (j = 1, 2, ..., max_harmonics)
    # A falha de pista externa gera picos em Z * fc, 2Z * fc, etc.
    for j in range(1, max_harmonics + 1):
        
        # --- Passo A: Frequência do Harmônico ---
        # Frequência de interesse: j-ésimo harmônico da BPFO
        f_harmonic_Hz = j * Z * fc_Hz
        w_harmonic_rad = 2 * np.pi * f_harmonic_Hz
        
        # O índice 's' para buscar o coeficiente de Fourier do pulso é Z*j
        s = Z * j
        
        # --- Passo B: Força de Excitação ---
        # Coeficiente do pulso para este harmônico
        F_zj = func_Fs(s)
        
        # Força Base = P_max * Z * F_zj
        # P_max é usado pois a falha está no centro da zona de carga (xi = 0)
        force_component = P_max * Z * F_zj
        
        # --- Passo C: Receptância (Somatório dos Modos) ---
        # Eq 20: Soma [ Xi(xi) / (Mi * wi^2) ]
        # Para xi=0, Xi(xi) = 1.
        sum_receptance = 0.0
        
        for _, row in df_outer.iterrows():
            wi = row['Freq_rad_s']
            Mi = row['Mass_kg']
            
            # Adiciona a contribuição deste modo (1 / k_dinamico)
            # Proteção contra divisão por zero, embora wi deva ser > 0
            if wi > 0:
                sum_receptance += 1.0 / (Mi * (wi**2))
        
        # --- Passo D: Deslocamento (Y) ---
        Y_displacement_m = force_component * sum_receptance
        
        # --- Passo E: Aceleração (A) ---
        # A = Y * w^2 (m/s²)
        accel_ms2 = Y_displacement_m * (w_harmonic_rad**2)
        
        freqs_hz.append(f_harmonic_Hz)
        ampls_accel.append(accel_ms2)
        
    # Retorna um DataFrame pronto para plotagem ou soma com sinal real
    return pd.DataFrame({
        'Harmonic_Order': range(1, max_harmonics + 1),
        'Frequency_Hz': freqs_hz,
        'Amplitude_m_s2': ampls_accel
    })

# --- Exemplo de Uso ---
# Definindo parâmetros (ex: defeito de 0.007" ~ 0.1778mm)
diametro_falha = 0.1778 
rotacao = 1750

df_espectro = calcular_espectro_outer_race(diametro_falha, rotacao)

print(f"--- Espectro Sintético (Outer Race, {rotacao} RPM) ---")
print(df_espectro)

--- Espectro Sintético (Outer Race, 1750 RPM) ---
   Harmonic_Order  Frequency_Hz  Amplitude_m_s2
0               1    104.563060        0.592869
1               2    209.126120        2.260964
2               3    313.689181        4.688199
3               4    418.252241        7.397343
4               5    522.815301        9.817893
5               6    627.378361       11.368884
6               7    731.941421       11.546157
7               8    836.504481       10.002860
8               9    941.067542        6.612566
9              10   1045.630602        1.506241


### Inner race

In [56]:
import numpy as np
import pandas as pd

def calcular_amplitude_inner_eq28_avancada(
    fault_diameter_mm, 
    rpm, 
    max_harmonics=5,     # Quantos picos principais (j) queremos calcular
    max_s_iter=50        # Até qual harmônico do pulso (s) vamos somar
):
    """
    Calcula a amplitude dos picos principais (BPFI) usando a interpretação da Eq. 28
    onde há uma somatória sobre os harmônicos 's' do pulso, modulando a receptância
    nos modos de índice (Zj) e (Zj +/- s).
    
    Fórmula do Usuário:
    Y_{Zj} = (Z*Po*Fo)/(M_{Zj}*w_{Zj}^2) + Sum_s [ (Z*Po*Fs) / (2 * M_{Zj +/- s} * w_{Zj +/- s}^2) ]
    """
    
    # 1. Obter coeficientes de Tandon
    tandon_data = calculate_tandon_coefficients(fault_diameter_mm, rpm, fault_type='inner')
    Po = tandon_data['Po']
    Fo = tandon_data['Fo']
    func_Fs = tandon_data['Fs']
    
    freqs_info = tandon_data['frequencies']
    bpfi_hz = freqs_info['defect_freq_hz']
    
    # 2. Obter Frequências Naturais (Assume-se que existam muitos modos calculados, ex: 100)
    # A função original retorna 6 modos. Vamos simular/extrapolar ou usar o que tiver.
    # Se você já rodou sua função ajustada para 100 modos, o DataFrame terá ~100 linhas.
    df_nat_freq = get_bearing_natural_frequencies() 
    df_inner = df_nat_freq[df_nat_freq['Race'] == 'Inner'].reset_index(drop=True)
    
    # Criar um lookup para facilitar acesso por índice (Modo 2 corresponde ao índice 2?)
    # A equação usa índices Zj. Vamos assumir que índice k mapeia para a linha k do DF (ou Modo k).
    # Ajuste: O Modo i começa em 2 na função original. Vamos mapear Indice -> Dados.
    dict_modes = {}
    for _, row in df_inner.iterrows():
        idx = int(row['Mode_i']) # Índice do modo
        dict_modes[idx] = {
            'M': row['Mass_kg'],
            'w2': row['Freq_rad_s']**2
        }
    
    Z = 9  # Número de esferas
    results = []
    
    # Função auxiliar para obter Receptância (1 / M*w^2) dado um índice k
    def get_receptance(k):
        if k in dict_modes and dict_modes[k]['w2'] > 0:
            return 1.0 / (dict_modes[k]['M'] * dict_modes[k]['w2'])
        else:
            return 0.0 # Se o modo não existe (índice fora da faixa calculada), retorna 0

    # 3. Loop pelos Harmônicos Principais (j)
    for j in range(1, max_harmonics + 1):
        
        f_peak_hz = j * bpfi_hz
        w_peak_rad = 2 * np.pi * f_peak_hz
        
        # Índice Base Z*j
        idx_base = Z * j
        
        # --- Termo 1: Componente DC (F_o) ---
        # Y_dc = (Z * Po * Fo) * Receptancia[Zj]
        recept_base = get_receptance(idx_base)
        term_1 = (Z * Po * Fo) * recept_base
        
        # --- Termo 2: Somatório sobre s (F_s) ---
        term_2_sum = 0.0
        
        # Iterar s de 1 até o limite definido (ou até acabar os modos)
        for s in range(1, max_s_iter + 1):
            Fs = func_Fs(s)
            
            # Índices Zj + s e Zj - s
            idx_plus = idx_base + s
            idx_minus = idx_base - s
            
            # Obter receptâncias para esses índices
            recept_plus = get_receptance(idx_plus)
            recept_minus = get_receptance(idx_minus)
            
            # Pela notação "Zj +/- s", somamos as contribuições de ambos os lados?
            # A fórmula tem "Sum ... / 2 ...". Vamos aplicar o fator e somar ambos.
            # Contribuição = (Z * Po * Fs / 2) * (Recept_Plus + Recept_Minus)
            term_s = (Z * Po * Fs / 2.0) * (recept_plus + recept_minus)
            
            term_2_sum += term_s
            
        # Amplitude Total de Deslocamento Y
        Y_total = term_1 + term_2_sum
        
        # Converter para Aceleração
        Acc_total = Y_total * (w_peak_rad**2)
        
        results.append({
            'Harmonic_Order': j,
            'Frequency_Hz': f_peak_hz,
            'Amplitude_Accel_m_s2': abs(Acc_total),
            'Terms_Debug': f"DC={term_1:.2e}, SumAC={term_2_sum:.2e}"
        })
        
    return pd.DataFrame(results)

# --- Exemplo de Uso ---
# Nota: Para isso funcionar bem, 'get_bearing_natural_frequencies' precisa retornar muitos modos.
# Como teste, usaremos o que tem, mas o ideal é sua versão com 100 modos.
df_eq28_custom = calcular_amplitude_inner_eq28_avancada(
    fault_diameter_mm=0.1778, 
    rpm=1750, 
    max_harmonics=10,
    max_s_iter= 87 # Soma até o 20º harmônico do pulso
)

print("--- Espectro Eq. 28 (User Custom) ---")
print(df_eq28_custom)

--- Espectro Eq. 28 (User Custom) ---
   Harmonic_Order  Frequency_Hz  Amplitude_Accel_m_s2  \
0               1    157.936940              0.049996   
1               2    315.873880              0.182498   
2               3    473.810819              0.344956   
3               4    631.747759              0.464206   
4               5    789.684699              0.470074   
5               6    947.621639              0.319622   
6               7   1105.558579              0.014962   
7               8   1263.495519              0.390977   
8               9   1421.432458              0.803248   
9              10   1579.369398              0.173546   

                    Terms_Debug  
0   DC=4.97e-11, SumAC=5.07e-08  
1   DC=3.02e-12, SumAC=4.63e-08  
2   DC=5.94e-13, SumAC=3.89e-08  
3   DC=1.88e-13, SumAC=2.95e-08  
4   DC=7.68e-14, SumAC=1.91e-08  
5   DC=3.70e-14, SumAC=9.02e-09  
6   DC=2.00e-14, SumAC=3.10e-10  
7  DC=1.17e-14, SumAC=-6.20e-09  
8  DC=7.30e-15, SumAC=-1.01e

In [57]:
import numpy as np
import pandas as pd

def calcular_sidebands_inner_eq_avancada(
    fault_diameter_mm, 
    rpm, 
    max_harmonics=3,     # j: 1, 2, 3...
    num_sidebands=3,     # r: 1, 2, 3...
    max_s_iter=50        # s: Limite da somatória interna
):
    """
    Calcula a amplitude dos SIDEBANDS (Upper e Lower) para falha de Pista Interna,
    baseado na fórmula fornecida pelo usuário (expansão da Eq. 29):
    
    Y = Termo_DC_Pulso + Termo_AC_Pulso
    
    Onde:
      Termo 1 (Fo): (Z * Pr * Fo) / (2 * M_{Zj-r} * w_{Zj-r}^2)
      Termo 2 (Fs): Sum_s [ (Z * Pr * Fs) / (4 * M_i * w_i^2) ]
                    com i = (Zj - r) +/- s
                    
    Nota: A função calcula para +r (Upper) e -r (Lower) tratando 'r' algebricamente.
    """
    
    # 1. Coeficientes Tandon
    tandon_data = calculate_tandon_coefficients(fault_diameter_mm, rpm, fault_type='inner')
    Pr_func = tandon_data['Pr']         # Função para harmônicos da carga P(r)
    Fo = tandon_data['Fo']              # Média do pulso
    func_Fs = tandon_data['Fs']         # Função para harmônicos do pulso F(s)
    
    freqs = tandon_data['frequencies']
    bpfi_hz = freqs['defect_freq_hz']
    f_shaft_hz = freqs['shaft_freq_hz'] # Frequência do eixo (ws)
    
    # 2. Modos Naturais (Assume-se que existam muitos modos calculados)
    df_nat_freq = get_bearing_natural_frequencies() 
    df_inner = df_nat_freq[df_nat_freq['Race'] == 'Inner'].reset_index(drop=True)
    
    # Mapeamento Modo -> Dados (Receptância)
    dict_modes = {}
    max_mode_idx = 0
    for _, row in df_inner.iterrows():
        idx = int(row['Mode_i'])
        # Armazena 1 / (M * w^2) para acesso rápido
        if row['Freq_rad_s'] > 0:
            dict_modes[idx] = 1.0 / (row['Mass_kg'] * (row['Freq_rad_s']**2))
        else:
            dict_modes[idx] = 0.0
        if idx > max_mode_idx: max_mode_idx = idx
            
    # Função auxiliar segura para receptância
    def get_receptance(k):
        # O índice físico é o valor absoluto |k|
        k_abs = abs(int(k))
        if k_abs in dict_modes:
            return dict_modes[k_abs]
        return 0.0 # Retorna 0 se o modo não foi calculado (ou usa max_mode_idx como fallback se preferir)

    Z = 9
    results = []
    
    # 3. Loops Principais
    # Loop j: Harmônicos da BPFO (Carrier)
    for j in range(1, max_harmonics + 1):
        
        # Loop r: Ordem dos Sidebands (1, 2, ... num_sidebands)
        # Vamos calcular para +r (Upper) e -r (Lower) explicitamente
        for r_abs in range(1, num_sidebands + 1):
            
            # Precisamos do coeficiente de carga Pr
            # Assumimos simetria: P(-r) = P(r)
            Pr = Pr_func(r_abs)
            
            # Calcular para Lower (-r) e Upper (+r)
            for sideband_sign in [-1, 1]:
                
                # r assinado
                r = r_abs * sideband_sign
                
                # Frequência do Sideband: j*BPFI + r*RPM
                # (Se r for negativo, é j*BPFI - |r|*RPM)
                f_sb_hz = (j * bpfi_hz) + (r * f_shaft_hz)
                
                # Se a frequência for negativa (possível se r for muito grande), ignoramos
                if f_sb_hz <= 0:
                    continue
                    
                w_sb_rad = 2 * np.pi * f_sb_hz
                
                # --- Índice Base para Receptância (Zj - r) ---
                # Na fórmula do usuário: M'_{Zj - r}. 
                # Nota: Se r for negativo (sideband inferior?), a fórmula escrita foi "+ r ws".
                # Se usarmos a fórmula genérica com 'r' algébrico:
                # O índice base "k0" segue o termo de frequência? 
                # A fórmula diz: Y_{Zj(...) + r ws}. O índice no denominador é Zj - r.
                # Isso sugere uma relação de convolução (diferença de índices).
                # Vamos manter estritamente: Indice = Z*j - r
                idx_base = (Z * j) - r  
                
                # --- TERMO 1: Contribuição DC do Pulso (Fo) ---
                # Y1 = (Z * Pr * Fo) / (2 * M_{Zj-r} * w^2)
                # Receptância no índice base
                recept_1 = get_receptance(idx_base)
                Y_term1 = (Z * Pr * Fo / 2.0) * recept_1
                
                # --- TERMO 2: Contribuição AC do Pulso (Somatório Fs) ---
                # Sum_s [ (Z * Pr * Fs / 4) * delta(...) ]
                # Delta implica somar modos i = (Zj - r) +/- s
                Y_term2_sum = 0.0
                
                for s in range(1, max_s_iter + 1):
                    Fs = func_Fs(s)
                    
                    # Índices determinados pela Delta de Dirac:
                    # i1 = (Zj - r) - s
                    # i2 = (Zj - r) + s
                    i_minus = idx_base - s
                    i_plus  = idx_base + s
                    
                    recept_minus = get_receptance(i_minus)
                    recept_plus  = get_receptance(i_plus)
                    
                    # Soma das contribuições
                    term_s = (Z * Pr * Fs / 4.0) * (recept_minus + recept_plus)
                    Y_term2_sum += term_s
                
                # --- Resultado Total ---
                Y_total = Y_term1 + Y_term2_sum
                
                # Converter para Aceleração
                Acc_total = abs(Y_total * (w_sb_rad**2))
                
                # Classificação para o DF
                sb_type = "Upper" if r > 0 else "Lower"
                
                results.append({
                    'Harmonic_j': j,
                    'Sideband_r': r_abs,
                    'Type': sb_type,
                    'Frequency_Hz': f_sb_hz,
                    'Amplitude_Accel_m_s2': Acc_total,
                    'Debug_Idx_Base': idx_base
                })
                
    # Retornar DataFrame ordenado
    df = pd.DataFrame(results)
    df = df.sort_values(by='Frequency_Hz').reset_index(drop=True)
    return df

# --- Teste da Função ---
# Verificando com os parâmetros atuais
df_sb_test = calcular_sidebands_inner_eq_avancada(
    fault_diameter_mm=0.1778, 
    rpm=1750, 
    max_harmonics=2, 
    num_sidebands=2,
    max_s_iter=20
)

print("--- Sidebands (Eq. Usuário) ---")
print(df_sb_test[['Harmonic_j', 'Type', 'Sideband_r', 'Frequency_Hz', 'Amplitude_Accel_m_s2']])

--- Sidebands (Eq. Usuário) ---
   Harmonic_j   Type  Sideband_r  Frequency_Hz  Amplitude_Accel_m_s2
0           1  Lower           2     99.603606              0.016997
1           1  Lower           1    128.770273              0.053682
2           1  Upper           1    187.103606              0.115203
3           1  Upper           2    216.270273              0.082807
4           2  Lower           2    257.540546              0.052159
5           2  Lower           1    286.707213              0.122926
6           2  Upper           1    345.040546              0.345450
7           2  Upper           2    374.207213              0.223465


## Cálculo dos atributos

## Salva CSV

In [13]:
# =============================================================================
# BLOCO 4: SALVAR OS DATAFRAMES FINAIS EM CSV (COM CAMINHO ESPECÍFICO)
# =============================================================================

import os

# Caminho de saída EXATO fornecido por você
caminho_base_output = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset\TCC'

# Cria o diretório se ele não existir
if not os.path.exists(caminho_base_output):
    os.makedirs(caminho_base_output)
    print(f"Diretório criado em: {caminho_base_output}")

# Define os nomes dos arquivos
caminho_csv_treino = os.path.join(caminho_base_output, 'df_treino_features.csv')
caminho_csv_teste = os.path.join(caminho_base_output, 'df_teste_features.csv')

try:
    # Salva o DataFrame de treino
    df_treino.to_csv(caminho_csv_treino, index=False)
    print(f"\nDataFrame de TREINO (features) salvo com sucesso em:")
    print(f"{caminho_csv_treino}")

    # Salva o DataFrame de teste
    df_teste.to_csv(caminho_csv_teste, index=False)
    print(f"\nDataFrame de TESTE (features) salvo com sucesso em:")
    print(f"{caminho_csv_teste}")

except NameError:
    print("\nERRO: Os DataFrames 'df_treino' ou 'df_teste' não foram encontrados.")
    print("Certifique-se de que o Bloco 3 foi executado com sucesso antes de salvar.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado ao salvar os arquivos: {e}")


DataFrame de TREINO (features) salvo com sucesso em:
C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset\TCC\df_treino_features.csv

DataFrame de TESTE (features) salvo com sucesso em:
C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset\TCC\df_teste_features.csv
